# 이미지 보강(Image Augmentation)
* 훈련 데이터에 없는 이미지를 새롭게 만들어내서 훈련 데이터를 보강
* 새로운 이미지는 훈련 데이터의 이미지를 원본으로 삼고 일정한 변형을 가해서 만들어냄.
* 이미지를 가로로 뒤집거나, 약한 회전, 기울이거나, 일부 확대, 평행 이동시켜 다양한 이미지를 만들어내서 훈련 데이터의 표현력을 더 좋게 만들어냄 
* tf.keras에 이미지 보강 작업을 도와주는 ImageDataGenerator가 있음.

In [1]:
import tensorflow as tf
import numpy as np 
from tensorflow.keras.preprocessing.image import ImageDataGenerator